In [1]:
import sys
import warnings

import scrublet as scr
import scvi
import re
import wget
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import anndata
from anndata import AnnData

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()

sc.settings.set_figure_params(dpi=80)
%matplotlib inline

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Set new working directory
os.chdir('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124')
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

sys.setrecursionlimit(1000000000)
print(sys.getrecursionlimit())

Current working directory: /Users/eroglulab/240926_scRNA_seq_dataset_JJR
Current working directory: /Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124
1000000000


# Read in raw counts and combine into raw AnnData object (Ramirez et al. data)

In [ ]:
adata_J1 = sc.read_10x_mtx(
    "E:\output_J1/filtered_matrix/sensitivity_4/", # the directory with the `.mtx` file
    var_names="gene_symbols", # use gene symbols for the variable names (variables-axis index)
    cache=True, # write a cache file for faster subsequent reading
)
adata_J1.var_names_make_unique()

adata_J3 = sc.read_10x_mtx(
    "E:\output_J3/filtered_matrix/sensitivity_4/", # the directory with the `.mtx` file
    var_names="gene_symbols", # use gene symbols for the variable names (variables-axis index)
    cache=True, # write a cache file for faster subsequent reading
)
adata_J3.var_names_make_unique()

adata_R2 = sc.read_10x_mtx(
    "E:\output_R2/filtered_matrix/sensitivity_4/", # the directory with the `.mtx` file
    var_names="gene_symbols", # use gene symbols for the variable names (variables-axis index)
    cache=True, # write a cache file for faster subsequent reading
)
adata_R2.var_names_make_unique()

adata_R4 = sc.read_10x_mtx(
    "E:\output_R4/filtered_matrix/sensitivity_4/", # the directory with the `.mtx` file
    var_names="gene_symbols", # use gene symbols for the variable names (variables-axis index)
    cache=True, # write a cache file for faster subsequent reading
)
adata_R4.var_names_make_unique()


In [ ]:
# add some metadata
adata_J1.obs['condition']="mCherry OE"
adata_J1.obs['sample']="mCherry-1"

adata_J3.obs['condition']="mCherry OE"
adata_J3.obs['sample']="mCherry-2"

adata_R2.obs['condition']="Hevin OE"
adata_R2.obs['sample']="Hevin-1"

adata_R4.obs['condition']="Hevin OE"
adata_R4.obs['sample']="Hevin-2"


# merge into one object.
#adata = adata_J1.concatenate(adata_J3, adata_R2, adata_R4)
adata = anndata.concat([adata_J1, adata_J3, adata_R2, adata_R4], join = 'outer')
adata.write('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/Hevin_AnnData_Objects/jjr_scanpy_outer_join_raw.h5ad')

# Read in raw counts and combine into raw AnnData object (Hammond et al. data)

In [ ]:
directory = "/Users/eroglulab/Desktop/Jianhong_analysis/Scripts_data/GSE121654"
list = os.listdir(directory)
list = np.array(list)
list = list[np.char.endswith(list,'.dge.txt.gz')]
header = [re.sub('GSM.+?_','',i) for i in list]
sample = [re.sub('.dge.txt.gz','',i) for i in header]
dir_list = [re.sub('GSM','/Users/eroglulab/Desktop/Jianhong_analysis/Scripts_data/GSE121654/GSM', i) for i in list]

In [ ]:
condition = ['E14',
 'P4',
 'P100',
 'P4',
 'P100',
 'P100',
 'E14',
 'P4',
 'Old',
 'P5',
 'P100',
 'P100',
 'Old',
 'P100',
 'P5',
 'P5',
 'LPC',
 'P30',
 'P5',
 'SALINE',
 'E14',
 'Old',
 'P4',
 'LPC',
 'SALINE',
 'E14',
 'P100',
 'P5',
 'P30',
 'P100',
 'P5',
 'P30',
 'LPC',
 'E14',
 'E14',
 'E14',
 'P100',
 'E14',
 'P4',
 'P100',
 'Old',
 'P100',
 'P5',
 'P4',
 'P5',
 'P30',
 'SALINE']

In [ ]:
sex = ['M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'F',
 'M',
 'F',
 'M',
 'M',
 'M',
 'F',
 'F',
 'F',
 'M',
 'M',
 'F',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'F',
 'M',
 'F',
 'M',
 'M',
 'F',
 'M',
 'M',
 'F',
 'F',
 'F',
 'F',
 'M',
 'F',
 'F',
 'M',
 'F',
 'M',
 'F',
 'F',
 'M',
 'M']

In [ ]:
for i in range(len(dir_list)):
    adata_sample = sc.read_text(dir_list[i]).transpose()
    
    adata_sample.obs['sample']=sample[i]
    adata_sample.obs['condition']=condition[i]
    adata_sample.obs['sex']=sex[i]
    if i == 0:
        adata = adata_sample
    else:
        adata = anndata.concat([adata,adata_sample], join='outer')

In [ ]:
adata.X = np.nan_to_num(adata.X)
adata.write('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE121654_raw.h5ad')

# Read in raw counts and combine into raw AnnData object (Li et al. data)

In [ ]:
import GEOparse
gse = GEOparse.get_GEO(geo="GSE123025", destdir='/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEOparse/')
metadata = gse.phenotype_data

li_raw = sc.read_csv('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEO_data/GSE123025_Single_myeloid_1922_cells_processed_data.csv.gz')
li_raw = li_raw.transpose()

In [ ]:
metadata_to_add = ['title','characteristics_ch1.1.genotype', 'characteristics_ch1.2.tissue',
       'characteristics_ch1.3.age', 'characteristics_ch1.4.gate',
       'characteristics_ch1.5.plate_id', 'characteristics_ch1.6.well_id',
       'characteristics_ch1.7.pooled_library_names',
       'characteristics_ch1.8.total_counts',
       'characteristics_ch1.9.detected_genes',
       'characteristics_ch1.10.qc_total_counts',
       'characteristics_ch1.11.qc_detected_genes',
       'characteristics_ch1.12.qc_ercc_correlation',
       'characteristics_ch1.13.qc_all_3_criteria']

In [ ]:
for x in metadata_to_add:
    #new_name = re.sub(r'^.*?_ch', '', x)
    index = x.rfind('.')
    new_name = x[index+1:len(x)]
    if x == metadata_to_add[0]:
        meta = pd.DataFrame({new_name:metadata[x]})
    else:
        meta[new_name] = metadata[x]

In [ ]:
for x in meta.keys().to_list():
    li_raw.obs[x] = meta[x].to_list()

In [ ]:
gse2 = GEOparse.get_GEO(geo="GSE123022", destdir='/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEOparse/')
metadata2 = gse2.phenotype_data

li_raw2 = sc.read_csv('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEO_data/GSE123022_C57BL6J_TREM2KO_369_Microglia_processed_data.csv.gz')
li_raw2 = li_raw2.transpose()

In [ ]:
for x in metadata_to_add:
    #new_name = re.sub(r'^.*?_ch', '', x)
    index = x.rfind('.')
    new_name = x[index+1:len(x)]
    if x == metadata_to_add[0]:
        meta2 = pd.DataFrame({new_name:metadata2[x]})
    else:
        meta2[new_name] = metadata2[x]

In [ ]:
for x in meta2.keys().to_list():
    li_raw2.obs[x] = meta2[x].to_list()

In [ ]:
li_raw.obs['run'] = '1'
li_raw2.obs['run'] = '2'

In [ ]:
from scipy import sparse

sparse_X = sparse.csr_matrix(li_raw.X)
li_raw.X = sparse_X

sparse_X = sparse.csr_matrix(li_raw2.X)
li_raw2.X = sparse_X

In [ ]:
li_raw_total = anndata.concat([li_raw,li_raw2], join = 'outer')

li_raw_total.write('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE123025_GSE123022_raw.h5ad')

# Read in raw counts and combine into raw AnnData object (Keren-Shaul et al. data)

In [ ]:
export_path = '/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/ProjecTILs/250122_ProjecTILs_KerenShauletal-DAM_Lietal-PAM/AnnDataObjects/'

In [ ]:
metadata = pd.read_csv('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/GSE98969_Keren-Shauletal/GSE98969_experimental_design_f(2).csv')
directory = '/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/GSE98969_Keren-Shauletal/GSE98969_RAW/'
list = os.listdir(directory)
list = np.array(list)
list = list[np.char.startswith(list,'GSM')]
list = list[np.char.endswith(list,'.txt.gz')]
dir_list = [re.sub('GSM',directory+"GSM", i) for i in list]

for i in dir_list:
    header = re.sub(directory,'',i)
    header = re.sub('GSM.+?_','',header)
    sample = re.sub('.txt.gz','',header)
    adata_sample = sc.read_text(i).transpose()
    sample_metadata = metadata[metadata['Amp_batch_ID']==sample]
    list_of_metadata = sample_metadata.keys().to_list()
    
    for x in list_of_metadata:
        adata_sample.obs[x] = sample_metadata[x].to_list()
    
    if i == dir_list[0]:
        adata = adata_sample
    else:
        adata = anndata.concat([adata,adata_sample], join='outer')

In [ ]:
adata.write(export_path+'Keren-Shauletal_raw.h5ad')

# Read in raw counts and combine into raw AnnData object (Zhou et al. data)

In [ ]:
import GEOparse
gse_zhou = GEOparse.get_GEO(geo="GSE140511", destdir='/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEOparse/')
metadata_zhou = gse_zhou.phenotype_data

In [ ]:
directory1 = '/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEO_data/GSE140511_RAW/Dataset1/'
directory2 = '/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/GEO_data/GSE140511_RAW/Dataset2/'

In [ ]:
directory = directory1

filelist = os.listdir(directory)
ends = filelist

filelist = np.array(filelist)
filelist = filelist[np.char.startswith(filelist,'GSM')]
filelist = filelist[np.char.endswith(filelist,'matrix.mtx.gz')]
filelist = filelist.tolist()

ends = np.array(ends)
ends = ends[np.char.startswith(ends,'GSM')]
ends = [ re.sub(r'G.*_.*_','', x) for x in ends]
footer = np.unique(ends)
footer = footer.tolist()

for x in filelist:
    index = x.find('matrix')
    header = x[0:index]
    sample_info = re.sub(r'G.*?_','',header)
    sample_info = sample_info[:-1]

    #print(header)
    #print(sample_info)

    for y in footer:
        file = header+y
        filepath = directory+file

        if y == 'barcodes.tsv.gz':
            #print(filepath)
            barcodes = pd.read_csv(filepath,sep='\t',header=None)
        if y == 'features.tsv.gz':
            #print(filepath)
            genes = pd.read_csv(filepath,sep='\t',header=None)
            #print(genes[1][0:5])
        if y == 'matrix.mtx.gz':
            #print(filepath)
            sample = sc.read_mtx(filepath).transpose()

    sample.obs_names = barcodes[0].to_list()
    sample.var_names = genes[1].to_list()

    sample.obs['sample'] = sample_info
    sample.obs_names_make_unique()
    sample.var_names_make_unique()

    #print(sample)

    if x == filelist[0]:
        adata = sample
        #print(adata.n_obs)
    else:
        adata = anndata.concat([adata,sample], join = 'outer')
        #print(adata.n_obs)

print(adata)

In [ ]:
adata.obs['age']='P196'

adata_7months = adata
adata_7months

In [ ]:
directory = directory2

filelist = os.listdir(directory)
ends = filelist

filelist = np.array(filelist)
filelist = filelist[np.char.startswith(filelist,'GSM')]
filelist = filelist[np.char.endswith(filelist,'matrix.mtx.gz')]
filelist = filelist.tolist()

ends = np.array(ends)
ends = ends[np.char.startswith(ends,'GSM')]
ends = [ re.sub(r'G.*_.*_','', x) for x in ends]
footer = np.unique(ends)
footer = footer.tolist()

for x in filelist:
    index = x.find('matrix')
    header = x[0:index]
    sample_info = re.sub(r'G.*?_','',header)
    sample_info = sample_info[:-1]

    #print(header)
    #print(sample_info)

    for y in footer:
        file = header+y
        filepath = directory+file

        if y == 'barcodes.tsv.gz':
            #print(filepath)
            barcodes = pd.read_csv(filepath,sep='\t',header=None)
        if y == 'genes.tsv.gz':
            #print(filepath)
            genes = pd.read_csv(filepath,sep='\t',header=None)
            print(genes[1][0:5])
        if y == 'matrix.mtx.gz':
            #print(filepath)
            sample = sc.read_mtx(filepath).transpose()

    sample.obs_names = barcodes[0].to_list()
    sample.var_names = genes[1].to_list()

    sample.obs['sample'] = sample_info
    sample.obs_names_make_unique()
    sample.var_names_make_unique()

    #print(sample)

    if x == filelist[0]:
        adata = sample
        #print(adata.n_obs)
    else:
        adata = anndata.concat([adata,sample], join = 'outer')
        #print(adata.n_obs)

#print(adata)

In [ ]:
adata.obs['age'] = 'P420'
adata_15months = adata
adata_15months

In [ ]:
adata_zhou = anndata.concat([adata_7months,adata_15months],join='outer')
adata_zhou.obs_names_make_unique()
adata_zhou.write('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE140511_raw.h5ad')

# Read in raw counts and combine into raw AnnData object (Escoubas et al. data)

In [ ]:
escoubas_raw = sc.read_csv('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/GSE173173_Escoubasetal/GSE173173_Dorman_P5_P7MG_counts.csv.gz')
escoubas_raw = escoubas_raw.transpose()

escoubas_metadata = pd.read_csv('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/GSE173173_Escoubasetal/GSE173173_Dorman_P5_P7MG_metadata_clustering.csv.gz')

In [ ]:
escoubas_raw.obs['age'] = escoubas_metadata['age'].to_numpy()
escoubas_raw.obs['sex'] = escoubas_metadata['sex'].to_numpy()
escoubas_raw.obs['condition'] = escoubas_metadata['condition'].to_numpy()
escoubas_raw.obs['sample'] = escoubas_metadata['fulldescription'].to_numpy()
escoubas_raw.obs['cluster'] = escoubas_metadata['finalclusters'].to_numpy()

In [ ]:
escoubas_raw.write('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE173173_raw.h5ad')

# Combine all datasets used for analysis

In [ ]:
adata_hev = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/Hevin_AnnData_Objects/jjr_scanpy_outer_join_raw.h5ad')
adata_li = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE123025_GSE123022_raw.h5ad')
adata_ham = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE121654_raw.h5ad')
adata_ker = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE98969_raw.h5ad')
adata_esc = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE173173_raw.h5ad')
adata_zho = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/GSE140511_raw.h5ad')

In [ ]:
adata_ham.var_names = adata_ham.var.index.str.replace('.','-') # The Hammond dataset is the only that seems to use 'mt.' instead of 'mt-' for mitochondrial genes.

In [ ]:
adata_hev.obs['dataset'] = 'Ramirezetal'
adata_li.obs['dataset'] = 'Lietal'
adata_ham.obs['dataset'] = 'Hammondetal'
adata_ker.obs['dataset'] = 'Keren-Shauletal'
adata_esc.obs['dataset'] = 'Escoubasetal'
adata_zho.obs['dataset'] = 'Zhouetal'

In [ ]:
adata_hev.obs_names_make_unique()
adata_li.obs_names_make_unique()
adata_ham.obs_names_make_unique()
adata_ker.obs_names_make_unique()
adata_esc.obs_names_make_unique()
adata_zho.obs_names_make_unique()

adata_hev.var_names_make_unique()
adata_li.var_names_make_unique()
adata_ham.var_names_make_unique()
adata_ker.var_names_make_unique()
adata_esc.var_names_make_unique()
adata_zho.var_names_make_unique()

In [ ]:
#This concatenates the data to make sure all datasets have the same genes listed even though they don't all have counts for all of them.

adata_concat = anndata.concat([adata_hev, adata_li, adata_ham, adata_ker, adata_esc, adata_zho], join = 'outer')

In [ ]:
adata_concat.obs_names_make_unique()

In [ ]:
#Setting a filter to remove cells that across all datasets have less than 200 genes annotated
# Setting a filter to remove genes that across all datasets don't have more than 3 cells
sc.pp.filter_cells(adata_concat, min_genes=200)
sc.pp.filter_genes(adata_concat, min_cells=3)

In [ ]:
#This separates the data back out to their respective datasets to do the quality control steps separately.

adata_hev = adata_concat[adata_concat.obs['dataset'] == 'Ramirezetal']
adata_li = adata_concat[adata_concat.obs['dataset'] == 'Lietal']
adata_ham = adata_concat[adata_concat.obs['dataset'] == 'Hammondetal']
adata_ker = adata_concat[adata_concat.obs['dataset'] == 'Keren-Shauletal']
adata_esc = adata_concat[adata_concat.obs['dataset'] == 'Escoubasetal']
adata_zho = adata_concat[adata_concat.obs['dataset'] == 'Zhouetal']

# Observation variables to include in every dataset:
#### 1. Condition = Experimental condition for the experiment
#### 2. Age = Age (in days) of the animal from which the cell was derived
#### 3. Sample = Identifier for the experiment to indicate biological replicates
#### 4. Genotype = Genotypes relevant for the experiment
#### 5. Batch = Sequencing batch (either given or inferred) used to train SCVI integration model
#### 6. Dataset = Dataset of origin for the cells
#### 7. Cluster = Clusters relevant for the previous experiment

# Initial QC and filtering for Ramirez et al. data

In [ ]:
adata_hev.obs['genotype'] = 'WT'
adata_hev.obs['age'] = 'P15'
adata_hev.obs['batch'] = 'batch0'

In [ ]:
adata_hev.var['mt'] = adata_hev.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_hev, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_hev, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata_hev, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_hev, x='total_counts', y='n_genes_by_counts')

adata_hev = adata_hev[adata_hev.obs.n_genes_by_counts < 4000, :]
adata_hev = adata_hev[adata_hev.obs.pct_counts_mt < 7.5, :]
adata_hev = adata_hev[adata_hev.obs.total_counts < 10000, :]

# Initial QC and filtering for Hammond et al. data

In [ ]:
sample_info = adata_ham.obs['sample'].to_list()
age = [re.sub(r'_.*','',x) for x in sample_info]
adata_ham.obs['age'] = age

In [ ]:
adata_ham.obs['genotype'] = 'WT'

In [ ]:
# These batch assignments were made by inferring batch effects after plotting each sample with its condition

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['E14_F_B10','E14_F_B12','E14_F_B6','E14_F_C1','E14_M_B11','E14_M_B7','E14_M_B8','E14_M_B9'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch1'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P4_F_A6','P4_M_A4','P4_M_A5'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch2'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P4_F_B3','P4_F_B4','P4_M_B5'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch3'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P5_F_A2', 'P5_M_A1'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch4'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P30_Male_1','P30_Male_2'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch5'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P30_male_3','P30_male_4'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch6'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P100_female_1','P100_female_2','P100_Male_1', 'P100_Male_2'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch7'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P100_female_3','P100_female_4','P100_male_3', 'P100_male_4'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch8'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P100_M_A1','P100_M_A2',])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch9'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P100_M_B5', ])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch10'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['Old_male_1','Old_male_2'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch11'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['Old_male_3','Old_male_4'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch12'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P100_M_LPC_A6','P100_M_SALINE_A3','P100_M_LPC_A4','P100_M_SALINE_A5'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch13'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P100_M_LPC_B10','P100_M_SALINE_B9'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch14'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(['P5_female_nopercoll_1', 'P5_female_nopercoll_2', 'P5_female_nopercoll_3','P5_female_percoll_1', 'P5_female_percoll_2', 'P5_female_percoll_3'])].to_list()
adata_ham.obs.loc[indices,'batch'] = 'batch22'

In [ ]:
adata_ham.obs['condition'] = adata_ham.obs['condition'].cat.set_categories(
    ['E14', 'LPC', 'Old', 'P4', 'P5', 'P30', 'P100', 'SALINE','nopercoll','percoll'])

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P5_female_nopercoll_3','P5_female_nopercoll_1','P5_female_nopercoll_2'])].to_list()
adata_ham.obs.loc[indices,'condition'] = 'nopercoll'

indices = adata_ham.obs.index[adata_ham.obs['sample'].isin(
    ['P5_female_percoll_2','P5_female_percoll_1','P5_female_percoll_3'])].to_list()
adata_ham.obs.loc[indices,'condition'] = 'percoll'

In [ ]:
adata_ham.var['mt'] = adata_ham.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_ham, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_ham, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata_ham, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_ham, x='total_counts', y='n_genes_by_counts')

adata_ham = adata_ham[adata_ham.obs.n_genes_by_counts < 2500, :]
adata_ham = adata_ham[adata_ham.obs.pct_counts_mt < 2.5, :]
adata_ham = adata_ham[adata_ham.obs.total_counts < 8000, :]

# Initial QC and filtering for Li et al. data

In [ ]:
adata_li = adata_li[adata_li.obs['qc_all_3_criteria']=='Y']

In [ ]:
adata_li.obs['sample'] = adata_li.obs['tissue']
adata_li.obs['condition'] = adata_li.obs['genotype']

In [ ]:
adata_li.obs['genotype'] = adata_li.obs['genotype'].cat.set_categories(
    ['TREM2_KO', 'wild type', 'WT'])

indices = adata_li.obs.index[adata_li.obs['genotype'].isin(
    ['wild type'])].to_list()
adata_li.obs.loc[indices,'genotype'] = 'WT'

In [ ]:
indices = adata_li.obs.index[adata_li.obs['run'].isin(
    ['1'])].to_list()
adata_li.obs.loc[indices,'batch'] = 'batch15'

indices = adata_li.obs.index[adata_li.obs['run'].isin(
    ['2'])].to_list()
adata_li.obs.loc[indices,'batch'] = 'batch16'

In [ ]:
adata_li.var['mt'] = adata_li.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_li, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_li, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata_li, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_li, x='total_counts', y='n_genes_by_counts')

#Cell filtering is commented out because the original publication annotated what cells were used and there is 
#little to no risk of doublets because the cells were sorted to have only one cell per well of a 384 well plate

#adata_li = adata_li[adata_li.obs.n_genes_by_counts < 2500, :]
#adata_li = adata_li[adata_li.obs.pct_counts_mt < 2.5, :]
#adata_li = adata_li[adata_li.obs.total_counts < 8000, :]

# Initial QC and filtering for Keren-Shaul et al. data

In [ ]:
adata_ker.obs['condition'] = adata_ker.obs['Mouse_ID']

In [ ]:
adata_ker.obs['genotype'] = adata_ker.obs['Mouse_ID']

adata_ker.obs['genotype'] = adata_ker.obs['genotype'].cat.set_categories(
    ['5XFAD', 'C57BL/6', 'SOD1', 'Trem2KO', 'Trem2KO_5XFAD', 'WT'])

indices = adata_ker.obs.index[adata_ker.obs['genotype'].isin(
    ['C57BL/6'])].to_list()
adata_ker.obs.loc[indices,'genotype'] = 'WT'

In [ ]:
metadata_ker = adata_ker.obs['Batch_desc'].to_list()
age_ker = [re.sub(r'old_','',x) for x in metadata_ker]
age_ker = [re.sub(r'young_','',x) for x in age_ker]
age_ker = [re.sub(r'ALS_mouse._?','',x) for x in age_ker]
age_ker = [re.sub(r'ALS_control._?','',x) for x in age_ker]
age_ker = [re.sub(r' .*','',x) for x in age_ker]
age_ker = [re.sub(r'_.*','',x) for x in age_ker]
age_ker = [re.sub('Trem2','unknown',x) for x in age_ker]
age_ker = [re.sub('7w','P196',x) for x in age_ker]
age_ker = [re.sub('..1m','P28',x) for x in age_ker]
age_ker = [re.sub('..3m','P84',x) for x in age_ker]
age_ker = [re.sub('..6m','P168',x) for x in age_ker]
age_ker = [re.sub('..8m','P224',x) for x in age_ker]
age_ker = [re.sub('..9m','P252',x) for x in age_ker]
age_ker = [re.sub('20m','P560',x) for x in age_ker]
age_ker = [re.sub('d135','P135',x) for x in age_ker]
age_ker = [re.sub('d80','P80',x) for x in age_ker]
#age_ker = np.array(age_ker)
#print(np.unique(metadata_ker))
#print(np.unique(age_ker))
adata_ker.obs['age'] = age_ker

In [ ]:
indices = adata_ker.obs.index[adata_ker.obs['Seq_batch_ID'].isin(
    ['SB79', 'SB84', 'SB90', 'SB93', 'SB100', 'SB121', 'SB126'])].to_list()
adata_ker.obs.loc[indices,'batch'] = 'batch17'

indices = adata_ker.obs.index[adata_ker.obs['Seq_batch_ID'].isin(
    ['SB94'])].to_list()
adata_ker.obs.loc[indices,'batch'] = 'batch18'

In [ ]:
adata_ker.obs['sample'] = adata_ker.obs['Batch_desc']

In [ ]:
adata_ker.var['mt'] = adata_ker.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_ker, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_ker, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

adata_ker.var['ERCC'] = adata_ker.var_names.str.startswith('ERCC')
sc.pp.calculate_qc_metrics(adata_ker, qc_vars=['ERCC'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_ker, ['n_genes_by_counts', 'total_counts', 'pct_counts_ERCC'],
             jitter=0.4, multi_panel=True)

In [ ]:
# Because none of the other datasets used ERCC spike-in I tried to set the criteria and ~1 standard deviation from the mean:
print('mean: ',adata_ker.obs['pct_counts_ERCC'].mean())
print('std: ',adata_ker.obs['pct_counts_ERCC'].std())

In [ ]:
sc.pl.scatter(adata_ker, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_ker, x='total_counts', y='pct_counts_ERCC')
sc.pl.scatter(adata_ker, x='total_counts', y='n_genes_by_counts')

#Because there are no counts annotated for the mitochondrial genes detected, I will not use percent mitochondrial genes as a filter.

adata_ker = adata_ker[adata_ker.obs.n_genes_by_counts < 2000, :]
adata_ker = adata_ker[adata_ker.obs.pct_counts_ERCC < 45, :]
adata_ker = adata_ker[adata_ker.obs.total_counts < 8000, :]

# Initial QC and filtering for Escoubas et al. data

In [ ]:
adata_esc.obs['batch'] = 'batch19'

In [ ]:
adata_esc.obs['genotype'] = 'WT'

In [ ]:
adata_esc.var['mt'] = adata_esc.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_esc, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_esc, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata_esc, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_esc, x='total_counts', y='n_genes_by_counts')

adata_esc = adata_esc[adata_esc.obs.n_genes_by_counts < 6000, :]
adata_esc = adata_esc[adata_esc.obs.pct_counts_mt < 7.5, :]
adata_esc = adata_esc[adata_esc.obs.total_counts < 35000, :]

# Initial QC and filtering for Zhou et al. data

In [ ]:
genotype_info = adata_zho.obs['sample'].to_list()
genotype_info = [re.sub('Trem2_KO','Trem2KO',x) for x in genotype_info]
genotype_info = [re.sub('WT_5XFAD','5XFAD',x) for x in genotype_info]
genotype_info = [re.sub(r'Trem2KO_5XFAD.*','Trem2KO_5XFAD',x) for x in genotype_info]
genotype_info = [re.sub(r'5XFAD.*','5XFAD',x) for x in genotype_info]
genotype_info = [re.sub('Trem2KO_Hip','Trem2KO',x) for x in genotype_info]
genotype_info = [re.sub('Trem2KO_Cor','Trem2KO',x) for x in genotype_info]
genotype_info = [re.sub('Trem2KO_1','Trem2KO',x) for x in genotype_info]
genotype_info = [re.sub('Trem2KO_2','Trem2KO',x) for x in genotype_info]
genotype_info = [re.sub('Trem2KO_3','Trem2KO',x) for x in genotype_info]
genotype_info = [re.sub(r'WT.*','WT',x) for x in genotype_info]

adata_zho.obs['genotype'] = genotype_info

In [ ]:
condition_info = adata_zho.obs['sample'].to_list()
condition_info = [re.sub(r'_.$','',x) for x in condition_info]
condition_info = [re.sub('Trem2_KO','Trem2KO',x) for x in condition_info]

adata_zho.obs['condition'] = condition_info

### Clustering data to isolate microglia 
#### Identify putative batch effects in microglia

In [ ]:
test = adata_zho.copy()

test1 = test[test.obs['age']=='P196']
test2 = test[test.obs['age']=='P420']

In [ ]:
#test = test1
#test = test2
test.var['mt'] = test.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(test, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(test, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(test, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(test, x='total_counts', y='n_genes_by_counts')

test = test[test.obs.n_genes_by_counts < 5000, :]
test = test[test.obs.pct_counts_mt < 7.5, :]
test = test[test.obs.total_counts < 15000, :]

In [ ]:
test.layers['counts'] = test.X

test.raw = test

sc.pp.normalize_total(test, target_sum=1e4)
sc.pp.log1p(test)

In [ ]:
sc.pp.highly_variable_genes(test)
sc.pl.highly_variable_genes(test)
test = test[:, test.var.highly_variable]
sc.pp.scale(test)

sc.tl.pca(test)
sc.pl.pca(test, color='sample')

sc.pp.neighbors(test)
sc.tl.umap(test)
sc.pl.umap(test, color=['sample'])

In [ ]:
sc.pl.umap(test, color=['Itgam','P2ry12','Cx3cr1','Tmem119', 'Hexb'], color_map = 'Purples')

In [ ]:
import leidenalg
sc.tl.leiden(test, key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(test, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(test, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(test, key_added="leiden_res1", resolution=1.0)
sc.pl.umap(test, color = ["leiden_res0_1", "leiden_res0_25", "leiden_res0_5", "leiden_res1"], legend_loc = 'on data')

In [ ]:
sc.pl.umap(test, color = ["leiden_res0_25", 'Hexb','P2ry12'], groups = ['8','21'], palette = 'Accent')

In [ ]:
test_microglia = test[test.obs['leiden_res0_25'].isin(['8','21'])]

In [ ]:
micro_indices = test.obs.index[test.obs['leiden_res0_25'].isin(
    ['8','21'])].to_list()
test_micro = adata_zho[adata_zho.obs.index.isin(micro_indices)]

In [ ]:
print(np.sum(test_microglia.X))
print(np.sum(test_micro.X))

test_microglia.var['mt'] = test_microglia.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(test_microglia, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(test_microglia, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

test_micro.var['mt'] = test_micro.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(test_micro, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(test_micro, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(test_micro, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(test_micro, x='total_counts', y='n_genes_by_counts')

test_micro = test_micro[test_micro.obs.n_genes_by_counts < 2000, :]
test_micro = test_micro[test_micro.obs.pct_counts_mt < 1.5, :]
test_micro = test_micro[test_micro.obs.total_counts < 2500, :]

In [ ]:
test_micro.layers['counts'] = test_micro.X

test_micro.raw = test_micro

sc.pp.normalize_total(test_micro, target_sum=1e4)
sc.pp.log1p(test_micro)

In [ ]:
sc.pp.highly_variable_genes(test_micro)
sc.pl.highly_variable_genes(test_micro)
test_micro = test_micro[:, test_micro.var.highly_variable]
sc.pp.scale(test_micro)

sc.tl.pca(test_micro)
sc.pl.pca(test_micro, color='sample')

sc.pp.neighbors(test_micro)
sc.tl.umap(test_micro)
sc.pl.umap(test_micro, color=['sample'])

In [ ]:
list_samples = test_micro.obs['sample'].unique().to_list()

palette = {}

for x in test_micro.obs['sample'].unique().to_list():
    palette[x] = 'black'

for x in list_samples:
    sample = test_micro[test_micro.obs['sample'].isin([x])]
    sc.pl.umap(test_micro, color = ['sample'], groups = [x], palette = palette)

#### From this analysis, I think that there are likely two batches in this dataset separated by the naming convention for each sample and age of the sample.
#### Batch 1 : Genotype_Sample#
#### Batch 2 : Genotype_Tissue Type

## Subset raw data by cell indices to just get microglia 

In [ ]:
micro_indices = test.obs.index[test.obs['leiden_res0_25'].isin(
    ['8','21'])].to_list()
micro_zho = adata_zho[adata_zho.obs.index.isin(micro_indices)]

In [ ]:
micro_zho.obs['batch'] = np.nan

indices = micro_zho.obs.index[micro_zho.obs['age'].isin(['P196'])].to_list()
micro_zho.obs.loc[indices,'batch'] = 'batch20'

indices = micro_zho.obs.index[micro_zho.obs['age'].isin(['P420'])].to_list()
micro_zho.obs.loc[indices,'batch'] = 'batch21'

In [ ]:
micro_zho.var['mt'] = micro_zho.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(micro_zho, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(micro_zho, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(micro_zho, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(micro_zho, x='total_counts', y='n_genes_by_counts')

micro_zho = micro_zho[micro_zho.obs.n_genes_by_counts < 2000, :]
micro_zho = micro_zho[micro_zho.obs.pct_counts_mt < 1.5, :]
micro_zho = micro_zho[micro_zho.obs.total_counts < 2500, :]

# Re-concatenate datasets after annotation and initial filtering

In [ ]:
adata_concat_filter = anndata.concat([micro_zho, adata_esc, adata_ker, adata_li, adata_ham, adata_hev])

# Remove variable names from the ERCC spike-in

## Datasets that have counts for ERCC spike in:
#### Keren-Shaul et al. 
#### Li et al. 

In [ ]:
adata_concat_filter.var['ERCC'] = adata_concat_filter.var_names.str.startswith('ERCC')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_concat_filter, qc_vars=['ERCC'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_concat_filter, ['n_genes_by_counts', 'total_counts', 'pct_counts_ERCC'],
             jitter=0.4, multi_panel=True)

In [ ]:
adata_concat_filter = adata_concat_filter[:,adata_concat_filter.var['ERCC']==False]

# Final step: Identify functional clusters relevant for this experiment
#### I will separate cells by dataset and cluster.
#### Then I will use published marker genes to identify clusters of interest.
#### List of functional clusters: DAMs, PAMs, Type-I interferon responsive clusters
#### Thankfully for Escoubas et al. this was already provided for by the authors.
#### Clusters of interest summary: Escoubas8, Keren-Shaul2, Zhou1, Li2, and Ramirez1

In [ ]:
adata_concat_filter.var['ERCC'] = adata_concat_filter.var_names.str.startswith('ERCC')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_concat_filter, qc_vars=['ERCC'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_concat_filter, ['n_genes_by_counts', 'total_counts', 'pct_counts_ERCC'],
             jitter=0.4, multi_panel=True)

In [ ]:
escoubas = adata_concat_filter[adata_concat_filter.obs['dataset']=='Escoubasetal']
keren_shaul = adata_concat_filter[adata_concat_filter.obs['dataset']=='Keren-Shauletal']
zhou = adata_concat_filter[adata_concat_filter.obs['dataset']=='Zhouetal']
li = adata_concat_filter[adata_concat_filter.obs['dataset']=='Lietal']
hammond = adata_concat_filter[adata_concat_filter.obs['dataset']=='Hammondetal']
ramirez = adata_concat_filter[adata_concat_filter.obs['dataset']=='Ramirezetal']

In [ ]:
# Here the goal was to take the cluster data that Escoubas et al. included and make a new column that will be used by all datasets to annotate
# clusters of interest.
escoubas_cluster_ID = escoubas.obs['cluster'].unique().tolist()
escoubas_cluster_ID

escoubas.obs['functional_cluster'] = np.nan
for x in escoubas_cluster_ID:
    indices = escoubas.obs.index[escoubas.obs['cluster'].isin([x])].to_list()
    escoubas.obs.loc[indices,'functional_cluster'] = 'Escoubas'+str(x)[0]

escoubas.obs

In [ ]:
# Hammond et al. will function mostly as a developmental atlas, therefore I will set the functional clusters as unknown.
hammond.obs['functional_cluster'] = 'unknown'

In [ ]:
#### Next I will cluster the Hevin data
hevin = ramirez.copy()

hevin.var['mt'] = hevin.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(hevin, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(hevin, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(hevin, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(hevin, x='total_counts', y='n_genes_by_counts')

hevin.layers['counts'] = hevin.X

hevin.raw = hevin

sc.pp.normalize_total(hevin, target_sum=1e4)
sc.pp.log1p(hevin)

sc.pp.highly_variable_genes(hevin)
sc.pl.highly_variable_genes(hevin)
hevin = hevin[:, hevin.var.highly_variable]
sc.pp.scale(hevin)

sc.tl.pca(hevin)
sc.pl.pca(hevin, color='sample')

sc.pp.neighbors(hevin)
sc.tl.umap(hevin)
sc.pl.umap(hevin, color=['sample'])

In [ ]:
import leidenalg
sc.tl.leiden(hevin, key_added="leiden_res0_05", resolution=0.05)
sc.tl.leiden(hevin, key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(hevin, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(hevin, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(hevin, key_added="leiden_res1", resolution=1.0)
sc.pl.umap(hevin, color = ["leiden_res0_05","leiden_res0_1", "leiden_res0_25", "leiden_res0_5", "leiden_res1"], legend_loc = 'on data')
sc.pl.umap(hevin, color = ['Apoe'], color_map = 'Purples')

In [ ]:
hevin_cluster_ID = hevin.obs['leiden_res0_25'].unique().tolist()
hevin_cluster_ID

ramirez.obs['functional_cluster'] = np.nan
for x in hevin_cluster_ID:
    indices = hevin.obs.index[hevin.obs['leiden_res0_25'].isin([x])].to_list()
    ramirez.obs.loc[indices,'functional_cluster'] = 'Ramirez'+str(x)

ramirez.obs

In [ ]:
# Next will be to cluster Keren-Shaul et al. and label DAMs
dams_keren = keren_shaul.copy()

# I will use scanpy combat to account for the slight batch effect in these data prior to clustering.
dams_keren_combat = dams_keren.copy()

dams_keren_combat.var['mt'] = dams_keren_combat.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(dams_keren_combat, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(dams_keren_combat, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(dams_keren_combat, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(dams_keren_combat, x='total_counts', y='n_genes_by_counts')

dams_keren_combat.layers['counts'] = dams_keren_combat.X

dams_keren_combat.raw = dams_keren_combat

sc.pp.normalize_total(dams_keren_combat, target_sum=1e4)
sc.pp.log1p(dams_keren_combat)

sc.pp.highly_variable_genes(dams_keren_combat)
sc.pl.highly_variable_genes(dams_keren_combat)
dams_keren_combat = dams_keren_combat[:, dams_keren_combat.var.highly_variable]
sc.pp.scale(dams_keren_combat)

sc.pp.combat(dams_keren_combat, key='batch')

sc.tl.pca(dams_keren_combat)
sc.pl.pca(dams_keren_combat, color='batch')

sc.pp.neighbors(dams_keren_combat)
sc.tl.umap(dams_keren_combat)
sc.pl.umap(dams_keren_combat, color=['batch'])

In [ ]:
import leidenalg
sc.tl.leiden(dams_keren_combat, key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(dams_keren_combat, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(dams_keren_combat, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(dams_keren_combat, key_added="leiden_res1", resolution=1.0)
sc.pl.umap(dams_keren_combat, color = ["leiden_res0_1", "leiden_res0_25", "leiden_res0_5", "leiden_res1"], legend_loc = 'on data')
sc.pl.umap(dams_keren_combat, color = ['Apoe', 'Clec7a'], color_map = 'Purples')

In [ ]:
keren_cluster_ID = dams_keren_combat.obs['leiden_res0_25'].unique().tolist()

keren_shaul.obs['functional_cluster'] = np.nan
for x in keren_cluster_ID:
    indices = dams_keren_combat.obs.index[dams_keren_combat.obs['leiden_res0_25'].isin([x])].to_list()
    keren_shaul.obs.loc[indices,'functional_cluster'] = 'Keren-Shaul'+str(x)

keren_shaul.obs

#### Keren-Shaul2 is the DAM cluster. I will keep the current annotation and decide if I need an additional column to group this with the other DAM cluster in the Zhou dataset later. 

In [ ]:
# Next I will cluster Zhou et al. data, but I will only use batch20 because it has significantly more microglia.
dams_zhou = zhou.copy()

dams_zhou = dams_zhou[dams_zhou.obs['batch']=='batch20']

dams_zhou.var['mt'] = dams_zhou.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(dams_zhou, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(dams_zhou, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(dams_zhou, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(dams_zhou, x='total_counts', y='n_genes_by_counts')

dams_zhou.layers['counts'] = dams_zhou.X

dams_zhou.raw = dams_zhou

sc.pp.normalize_total(dams_zhou, target_sum=1e4)
sc.pp.log1p(dams_zhou)

sc.pp.highly_variable_genes(dams_zhou)
sc.pl.highly_variable_genes(dams_zhou)
dams_zhou = dams_zhou[:, dams_zhou.var.highly_variable]
sc.pp.scale(dams_zhou)

sc.tl.pca(dams_zhou)
sc.pl.pca(dams_zhou, color='sample')

sc.pp.neighbors(dams_zhou)
sc.tl.umap(dams_zhou)
sc.pl.umap(dams_zhou, color=['sample'])

In [ ]:
import leidenalg
sc.tl.leiden(dams_zhou, key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(dams_zhou, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(dams_zhou, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(dams_zhou, key_added="leiden_res1", resolution=1.0)
sc.pl.umap(dams_zhou, color = ["leiden_res0_1", "leiden_res0_25", "leiden_res0_5", "leiden_res1"], legend_loc = 'on data')
sc.pl.umap(dams_zhou, color = ['Apoe','Clec7a', 'Gpnmb'], color_map = 'Purples')

In [ ]:
zhou_cluster_ID = dams_zhou.obs['leiden_res0_25'].unique().tolist()

zhou.obs['functional_cluster'] = np.nan
for x in zhou_cluster_ID:
    indices = dams_zhou.obs.index[dams_zhou.obs['leiden_res0_25'].isin([x])].to_list()
    zhou.obs.loc[indices,'functional_cluster'] = 'Zhou'+str(x)

zhou_unknown_cluster_indices = zhou.obs.index[zhou.obs['batch']== 'batch21'].to_list()
zhou.obs.loc[zhou_unknown_cluster_indices,'functional_cluster'] = 'unknown'

zhou.obs

#### Zhou1 is the DAM cluster. I will keep the current annotation and decide if I need an additional column to group this with the other DAM cluster in the Keren-Shaul dataset later. 

In [ ]:
# Finally, I will cluster Li et al. but I will only use batch15 because it has significantly more cells
pams_li = li.copy()

pams_li = pams_li[pams_li.obs['batch']=='batch15']

pams_li.var['mt'] = pams_li.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(pams_li, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(pams_li, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(pams_li, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(pams_li, x='total_counts', y='n_genes_by_counts')

pams_li.layers['counts'] = pams_li.X

pams_li.raw = pams_li

sc.pp.normalize_total(pams_li, target_sum=1e4)
sc.pp.log1p(pams_li)

sc.pp.highly_variable_genes(pams_li)
sc.pl.highly_variable_genes(pams_li)
pams_li = pams_li[:, pams_li.var.highly_variable]
sc.pp.scale(pams_li)

sc.tl.pca(pams_li)
sc.pl.pca(pams_li, color='sample')

sc.pp.neighbors(pams_li)
sc.tl.umap(pams_li)
sc.pl.umap(pams_li, color=['sample'])

In [ ]:
import leidenalg
sc.tl.leiden(pams_li, key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(pams_li, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(pams_li, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(pams_li, key_added="leiden_res1", resolution=1.0)
sc.pl.umap(pams_li, color = ["leiden_res0_1", "leiden_res0_25", "leiden_res0_5", "leiden_res1"], legend_loc = 'on data')
sc.pl.umap(pams_li, color = ['Apoe','Clec7a', 'Gpnmb','Spp1'], color_map = 'Purples')

In [ ]:
sc.tl.rank_genes_groups(pams_li, 'leiden_res1', method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(pams_li, groups = ['1','2','3','0','5','6'], groupby='leiden_res1', n_genes = 10)

In [ ]:
li_cluster_ID = pams_li.obs['leiden_res1'].unique().tolist()

li.obs['functional_cluster'] = np.nan
for x in li_cluster_ID:
    indices = pams_li.obs.index[pams_li.obs['leiden_res1'].isin([x])].to_list()
    li.obs.loc[indices,'functional_cluster'] = 'Li'+str(x)

li_unknown_cluster_indices = li.obs.index[li.obs['batch']== 'batch16'].to_list()
li.obs.loc[li_unknown_cluster_indices,'functional_cluster'] = 'unknown'

li.obs

#### Li2 is the PAM cluster. I will keep the current annotation and decide if I need an additional column to group this with the DAM clusters.

In [ ]:
escoubas.obs['functional_cluster']=escoubas.obs['functional_cluster'].astype('category')
hammond.obs['functional_cluster']=hammond.obs['functional_cluster'].astype('category')
ramirez.obs['functional_cluster']=ramirez.obs['functional_cluster'].astype('category')
keren_shaul.obs['functional_cluster']=keren_shaul.obs['functional_cluster'].astype('category')
zhou.obs['functional_cluster']=zhou.obs['functional_cluster'].astype('category')
li.obs['functional_cluster']=li.obs['functional_cluster'].astype('category')

In [ ]:
concat_final = anndata.concat([escoubas,hammond,ramirez,keren_shaul,zhou,li])

In [ ]:
concat_final.write('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/Concat_raw_annotated.h5ad')

# Export files for ProjecTILs in Seurat

In [ ]:
adata1 = sc.read_h5ad('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/Input_Anndata/Concat_raw_annotated.h5ad')

In [ ]:
adata = anndata.AnnData(adata1.X.toarray())
adata.obs_names = adata1.obs_names
adata.var_names = adata1.var_names

adata.obs['sample'] = adata1.obs['sample']
adata.obs['condition'] = adata1.obs['condition']
adata.obs['genotype'] = adata1.obs['genotype']
adata.obs['age'] = adata1.obs['age']
adata.obs['dataset'] = adata1.obs['dataset']
adata.obs['cluster'] = adata1.obs['functional_cluster']
adata.obs['batch'] = adata1.obs['batch']

In [ ]:
adata_hev = adata[adata.obs['condition']=='Hevin OE']
adata_mcherry = adata[adata.obs['condition']=='mCherry OE']
adata_ram = adata[adata.obs['dataset']=='Ramirezetal']
adata_ham = adata[adata.obs['dataset']=='Hammondetal']
adata_ker = adata[adata.obs['dataset']=='Keren-Shauletal']
adata_zho = adata[adata.obs['dataset']=='Zhouetal']
adata_li = adata[adata.obs['dataset']=='Lietal']
adata_esc = adata[adata.obs['dataset']=='Escoubasetal']

In [ ]:
adata_inf = adata_esc[adata_esc.obs['cluster'].isin(['Escoubas8'])]
adata_homeo = adata_esc[adata_esc.obs['cluster'].isin(['Escoubas0','Escoubas5'])]

adata_pam = adata_li[adata_li.obs['cluster'].isin(['Li2'])]
adata_non_pam = adata_li[adata_li.obs['cluster'].isin(['Li1','Li3'])]

adata_ker_batch17 = adata_ker[adata_ker.obs['batch']=='batch17']
adata_ker_wt = adata_ker_batch17[adata_ker_batch17.obs['genotype']=='WT']
adata_ker_ad = adata_ker_batch17[adata_ker_batch17.obs['genotype']=='5XFAD']

adata_zho_batch20 = adata_zho[adata_zho.obs['batch']=='batch20']
adata_zho_wt = adata_zho_batch20[adata_zho_batch20.obs['genotype']=='WT']
adata_zho_ad = adata_zho_batch20[adata_zho_batch20.obs['genotype']=='5XFAD']

adata_lpc = adata_ham[adata_ham.obs['condition']=='LPC']
adata_sal = adata_ham[adata_ham.obs['condition']=='SALINE']

adata_e14 = adata_ham[adata_ham.obs['condition']=='E14']
adata_p4_5 = adata_ham[adata_ham.obs['condition'].isin(['P4','P5'])]
adata_p30 = adata_ham[adata_ham.obs['condition']=='P30']
adata_p100 = adata_ham[adata_ham.obs['condition']=='P100']
adata_old = adata_ham[adata_ham.obs['condition']=='Old']

In [ ]:
export = '/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Compare_Outside_Data/SCVI_Integrate_public_datasets_250124/ProjectTILs/AnnData_Input/'
adata_inf.write(export+'Escoubas_Inf.h5ad')
adata_homeo.write(export+'Escoubas_Homeo.h5ad')

adata_pam.write(export+'Li_pams.h5ad')
adata_non_pam.write(export+'Li_nonpams.h5ad')

adata_ker_wt.write(export+'Keren-Shaul_WT.h5ad')
adata_ker_ad.write(export+'Keren-Shaul_AD.h5ad')

adata_zho_wt.write(export+'Zhou_WT.h5ad')
adata_zho_ad.write(export+'Zhou_AD.h5ad')

adata_lpc.write(export+'Hammond_LPC.h5ad')
adata_sal.write(export+'Hammond_Saline.h5ad')

adata_e14.write(export+'Hammond_E14.h5ad')
adata_p4_5.write(export+'Hammond_P4-5.h5ad')
adata_p30.write(export+'Hammond_P30.h5ad')
adata_p100.write(export+'Hammond_P100.h5ad')
adata_old.write(export+'Hammond_Old.h5ad')

In [ ]:
adata_hev.write(export+'Ramirez_Hevin.h5ad')
adata_mcherry.write(export+'Ramirez_mCherry.h5ad')
adata_ram.write(export+'Ramirez_reference.h5ad')

# Read in Ramirez et al. data post-Seurat

In [9]:
adata_hev = sc.read_csv('/Volumes/Argo II/SingleCell_GEO_Submission/Ramirez_normalized_counts.csv')
meta_data_hev = pd.read_csv('/Volumes/Argo II/SingleCell_GEO_Submission/Ramirez_metadata.csv')

In [10]:
adata_ram = adata_hev.transpose()

In [12]:
adata_ram.obs['sample'] = meta_data_hev['sample'].to_numpy()
adata_ram.obs['condition'] = meta_data_hev['condition'].to_numpy()
adata_ram.obs['genotype'] = meta_data_hev['genotype'].to_numpy()
adata_ram.obs['age'] = meta_data_hev['age'].to_numpy()
adata_ram.obs['dataset'] = meta_data_hev['dataset'].to_numpy()
adata_ram.obs['cluster'] = meta_data_hev['functional.cluster'].to_numpy()

In [13]:
marker_genes = [
    'P2ry12',
    'Itgam',
    'Tmem119',
    'Cx3cr1',
    'Hexb',
    'Tgfbr1',
    'Fcrls',
    'C1qa',
    'Trem2',
    
    
    'Lilra5',
    'Clec4n',
    'Ednrb',
    'Snx6',
    'Snx2',
    'Dab2',
    'Cd36',
    'Ap1b1',
    'Mrc1',
    'Cd163',
    
    'Clec10a',
    'Ptprc',
    'H2-Eb1',
    'H2-Aa',
    'Ccr2',
    'Cd209a',
    'Ace',
    'Nr4a1',
    
    'Ly6c1',
    'Ly6c2',
    'Pecam1',
    'Cldn5',
    'Slco1c1',
    'Ocln',

    'Sparcl1',
    'Slc1a2',
    'Slc1a3',
    'Ptprz1',
    'Nrxn1',
    'S100b',
    'Ncan',
    'Gja1',
    'Gjb6',
    'Sox9',
    'Ezr',
    'Aqp4',
]

In [ ]:
image_directory = '/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Figure_Drafts/250204/'
os.chdir(image_directory)

sc.pl.dotplot(
    adata_ram,
    categories_order = ['Cluster 0','Cluster 1','Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6'],
    groupby="cluster",
    var_names=marker_genes,
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
    cmap = 'viridis',
    save = 'Celltype_Markers_dotplot.pdf'
)

In [ ]:
sc.tl.rank_genes_groups(adata_ram, 'cluster', method='wilcoxon')
sc.pl.rank_genes_groups(adata_ram)

In [ ]:
result = adata_ram.uns["rank_genes_groups"]
groups = result["names"].dtype.names
grouped_results = pd.DataFrame(
    {
        group + "_" + key[:1]: result[key][group]
        for group in groups
        for key in ["names", "scores", "logfoldchanges", "pvals_adj"]
    }
)

grouped_results.to_csv('/Volumes/Argonaut/Hevin_OE_SingleCell_RNAseq/Figure_Drafts/250204/Ramirez_etal_Wilcoxon.csv')

In [ ]:
marker_genes_top5 = {
    "Cluster 0": ["Cst3","Sparc","P2ry12","Hexb","Selplg"],
    "Cluster 1": ["Apoe","Bst2","Pmepa1","Tmem176b","Gpr65"],
    "Cluster 2": ["Mrc1","F13a1","Cbr2","Dab2","Pf4"],
    "Cluster 3": ['Cldn5','Bsg','Flt1','Tsc22d1','Igfbp7'],
    "Cluster 4": ['S100a6','Crip1','Cd74','Plbd1','Lsp1'], 
    "Cluster 5": ['Hmgb2','Stmn1','H2az1','Hmgn2','Hmgn2-ps'],
    "Cluster 6": ['Ptn', 'Cpe', 'Atp1a2','Slc1a2','Sparcl1'],
}

In [ ]:
sc.pl.dotplot(
    adata_ram,
    categories_order = ['Cluster 0','Cluster 1','Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6'],
    groupby="cell_type",
    var_names=marker_genes_top5,
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
    cmap = 'viridis',
    save = "Top_5_marker_genes_by_cluster.pdf"
)

In [ ]:
sc.pl.dotplot(
    adata_ram,
    categories_order = ['Cluster 0','Cluster 1','Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6'],
    groupby="cell_type",
    var_names=['Tlr2','Tlr1','Ly96','Cd14','Tlr4'],
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
    cmap = 'viridis',
    save = "TLR_related_genes.pdf"
)

In [ ]:
marker_genes = ['Apoe','Cd63','Ctsb','Lyz2','Plek','Pld3','Igf1','Anxa5','Plin2','Ctsd','Cd9','Spp1','Lpl',
'Gpnmb',
'Hpse',
'Aplp2',
'Gpx3',
'Csf1',
'Lgals3',
]

sc.pl.dotplot(
    hevin,
    categories_order = ['Cluster 0','Cluster 1','Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6'],
    groupby="cell_type",
    var_names=marker_genes,
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
    cmap = 'viridis',
    save = "PAM_genes_allclusters.png"
)

In [ ]:
marker_genes = ['Apoe','Cd63','Ctsb','Lyz2','Plek','Pld3','Igf1','Anxa5','Plin2','Ctsd','Cd9','Spp1','Lpl',
'Gpnmb',
'Hpse',
'Aplp2',
'Gpx3',
'Csf1',
'Lgals3',
]

hevin_micro = hevin[hevin.obs['cell_type'].isin(['Cluster 0','Cluster 5','Cluster 1'])]
sc.pl.dotplot(
    hevin_micro,
    categories_order = ['Cluster 0','Cluster 5','Cluster 1'],
    groupby="cell_type",
    var_names=marker_genes,
    standard_scale="var",  # standard scale: normalize each gene to range from 0 to 1
    cmap = 'viridis',
    save = "PAM_genes_microgliaclusters.png"
)